In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/subjposaware/__huggingface_repos__.json
/kaggle/input/subjposaware/results/improved_pos_model/checkpoint-2100/trainer_state.json
/kaggle/input/subjposaware/results/improved_pos_model/checkpoint-2100/training_args.bin
/kaggle/input/subjposaware/results/improved_pos_model/checkpoint-2100/scaler.pt
/kaggle/input/subjposaware/results/improved_pos_model/checkpoint-2100/scheduler.pt
/kaggle/input/subjposaware/results/improved_pos_model/checkpoint-2100/model.safetensors
/kaggle/input/subjposaware/results/improved_pos_model/checkpoint-2100/optimizer.pt
/kaggle/input/subjposaware/results/improved_pos_model/checkpoint-2100/rng_state.pth
/kaggle/input/subjposaware/wandb/run-20250604_182537-5t1nx01t/run-5t1nx01t.wandb
/kaggle/input/subjposaware/wandb/run-20250604_182537-5t1nx01t/logs/debug.log
/kaggle/input/subjposaware/wandb/run-20250604_182537-5t1nx01t/logs/debug-internal.log
/kaggle/input/subjposaware/wandb/run-20250604_182537-5t1nx01t/files/wandb-summary.json
/kaggle/input/subjpo

In [2]:
!pip install bnlp-toolkit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; 

In [ ]:
from bnlp import BengaliPOS

bn_pos = BengaliPOS()
print(bn_pos.model.classes_)

['JJ', 'NC', 'PU', 'CCD', 'NP', 'VM', 'JQ', 'PRL', 'CX', 'DAB', 'PPR', 'CSB', 'PP', 'NV', 'CCL', 'AMN', 'RDS', 'VAUX', 'NST', 'ALC', 'PWH', 'RDF', 'PRF', 'PRC', 'LC', 'DRL', 'LV', 'DWH', 'CIN', 'RDX']


In [4]:
#config
RANDOM_SEED = 42
import random
import os
import torch

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True  # Slower but reproducible
    torch.backends.cudnn.benchmark = False     # Disable benchmarking to ensure determinism

    # Set seed in Transformers (for Trainer)
    from transformers import set_seed as hf_set_seed
    hf_set_seed(seed)

set_seed(42)

2025-06-19 20:35:02.797443: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750365302.972620      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750365303.029934      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
train_df = pd.read_excel("/kaggle/input/subjectivity-detection-splitted/Split80/train.xlsx", engine='openpyxl')
val_df   = pd.read_excel("/kaggle/input/subjectivity-detection-splitted/Split80/val.xlsx", engine='openpyxl')
test_df  = pd.read_excel("/kaggle/input/subjectivity-detection-splitted/Split80/test.xlsx", engine='openpyxl')


In [ ]:
from sklearn.preprocessing import LabelEncoder

def encode_labels(train_df, val_df, test_df):
    """
    Encodes the 'Label' column in train, val, and test DataFrames using LabelEncoder.
    Returns the updated DataFrames.
    """
    le = LabelEncoder()
    train_df['Label'] = le.fit_transform(train_df['Label'])
    val_df['Label']   = le.transform(val_df['Label'])
    test_df['Label']  = le.transform(test_df['Label'])
    return train_df, val_df, test_df


def prepare_datasets(train_df, val_df, test_df, transformerDL=True):
    """
    Extracts features and labels from the given DataFrames based on transformerDL flag.
    Returns: X_train, X_val, X_test, y_train, y_val, y_test
    """
    text_col = 'Text' if transformerDL else 'cleanedText'
    
    X_train = train_df[text_col]
    y_train = train_df['Label']
    
    X_val = val_df[text_col]
    y_val = val_df['Label']
    
    X_test = test_df[text_col]
    y_test = test_df['Label']
    
    return X_train, X_val, X_test, y_train, y_val, y_test
    
# Encode labels
train_df, val_df, test_df = encode_labels(train_df, val_df, test_df)

# Prepare datasets
X_train, X_val, X_test, y_train, y_val, y_test = prepare_datasets(train_df, val_df, test_df, transformerDL=True)

import wandb
wandb.login(key=wandb_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: minhazulkabir (minhazulkabir-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report

import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil
import gc
from transformers import AutoTokenizer, AutoModel, AutoConfig, Trainer, TrainingArguments, EarlyStoppingCallback
from transformers.modeling_outputs import SequenceClassifierOutput
from sklearn.metrics import accuracy_score, f1_score, classification_report
from bnlp import BengaliPOS
import torch.nn as nn
import torch.nn.functional as F

class POSTagEncoder:
    def __init__(self):
        self.pos_tagger = BengaliPOS()
        self.pos_to_id = {
            'PAD': 0, 'UNK': 1, 'CLS': 2, 'SEP': 3,
            'noun': 4, 'verb': 5, 'adjective': 6, 'adverb': 7,
            'pronoun': 8, 'preposition': 9, 'conjunction': 10,
            'interjection': 11, 'determiner': 12, 'punctuation': 13
        }
        self.bnlp_to_coarse = {
            'JJ': 'adjective', 'JQ': 'adjective',
            'NC': 'noun', 'NP': 'noun', 'NST': 'noun', 'NV': 'noun', 'LC': 'noun',
            'VM': 'verb', 'VAUX': 'verb', 'LV': 'verb',
            'PPR': 'pronoun', 'PRL': 'pronoun', 'PRF': 'pronoun', 'PRC': 'pronoun',
            'DAB': 'determiner', 'DRL': 'determiner', 'DWH': 'determiner',
            'CCD': 'conjunction', 'CCL': 'conjunction', 'CSB': 'conjunction',
            'AMN': 'adverb', 'RDS': 'adverb', 'RDF': 'adverb', 'RDX': 'adverb',
            'PWH': 'interjection',
            'CX': 'preposition', 'PP': 'preposition',
            'PU': 'punctuation',
            'ALC': 'UNK', 'CIN': 'UNK'
        }

    def tag_and_align(self, text, tokenizer, max_len):
        words = text.strip().split()
        try:
            tags = self.pos_tagger.tag(" ".join(words))
            coarse_tags = [self.bnlp_to_coarse.get(tag, 'UNK') for _, tag in tags]
        except:
            coarse_tags = ['UNK'] * len(words)

        encoding = tokenizer(
            text, max_length=max_len, padding="max_length",
            truncation=True, return_offsets_mapping=True, add_special_tokens=True
        )

        word_ids = encoding.word_ids()
        pos_ids = []
        for i, word_id in enumerate(word_ids):
            if word_id is None:
                if i == 0: pos_ids.append(self.pos_to_id['CLS'])
                elif encoding['input_ids'][i] == tokenizer.sep_token_id:
                    pos_ids.append(self.pos_to_id['SEP'])
                else: pos_ids.append(self.pos_to_id['PAD'])
            elif word_id < len(coarse_tags):
                pos_ids.append(self.pos_to_id.get(coarse_tags[word_id], self.pos_to_id['UNK']))
            else:
                pos_ids.append(self.pos_to_id['UNK'])
        return encoding, pos_ids
#models
model_configs = [
    {"name": "MuRIL", "model_name": "google/muril-base-cased"},
    {"name": "buetBangla-BERT", "model_name": "csebuetnlp/banglabert"},
    {"name": "XLM-Roberta", "model_name": "xlm-roberta-base"},
    {"name": "Multilingual BERT", "model_name": "bert-base-multilingual-cased"},
    {"name": "sagorBangla-BERT", "model_name": "sagorsarker/bangla-bert-base"},
    {"name": "DistilBERT Multilingual", "model_name": "distilbert-base-multilingual-cased"},
    {"name": "IndicBERT", "model_name": "ai4bharat/indic-bert"},
    {"name": "Bangla Electra", "model_name": "monsoon-nlp/bangla-electra"},
]

#Improved POS-Aware BanglaBERT
class ImprovedPOSAwareBanglaBERT(nn.Module):
    def __init__(self, model_name, num_labels=2, pos_vocab_size=14, pos_embed_dim=64):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        hidden_size = self.bert.config.hidden_size
        self.pos_embedding = nn.Embedding(pos_vocab_size, pos_embed_dim)
        self.pos_projection = nn.Linear(pos_embed_dim, hidden_size // 4)
        self.fusion_layer = nn.Linear(hidden_size + hidden_size // 4, hidden_size)
        self.layer_norm = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(hidden_size, num_labels)

        nn.init.xavier_uniform_(self.pos_embedding.weight)
        nn.init.xavier_uniform_(self.pos_projection.weight)
        nn.init.zeros_(self.pos_projection.bias)
        nn.init.xavier_uniform_(self.fusion_layer.weight)
        nn.init.zeros_(self.fusion_layer.bias)
        nn.init.xavier_uniform_(self.classifier.weight)
        nn.init.zeros_(self.classifier.bias)

    def forward(self, input_ids, attention_mask, pos_ids, token_type_ids=None, labels=None):
        bert_outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            #token_type_ids=token_type_ids,#works for AutoModelForSequenceClassification
            **({"token_type_ids": token_type_ids} if token_type_ids is not None and "token_type_ids" in self.bert.forward.__code__.co_varnames else {}),
            return_dict=True
        )
        sequence_output = bert_outputs.last_hidden_state
        pos_embeds = self.pos_embedding(pos_ids)
        pos_embeds = self.pos_projection(pos_embeds)
        combined = torch.cat([sequence_output, pos_embeds], dim=-1)
        fused_output = self.fusion_layer(combined)
        fused_output = F.gelu(fused_output)#FT
        #fused_output = F.relu(fused_output)
        fused_output = self.layer_norm(fused_output)
        cls_output = self.dropout(fused_output[:, 0, :])
        logits = self.classifier(cls_output)

        loss = None
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=bert_outputs.hidden_states,
            attentions=bert_outputs.attentions,
        )
        
#Dataset
class POSAwareDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_len, pos_encoder):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pos_encoder = pos_encoder

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding, pos_ids = self.pos_encoder.tag_and_align(self.texts[idx], self.tokenizer, self.max_len)
        item = {
            "input_ids": torch.tensor(encoding["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(encoding["attention_mask"], dtype=torch.long),
            "pos_ids": torch.tensor(pos_ids, dtype=torch.long),
        }
        if "token_type_ids" in encoding:
            item["token_type_ids"] = torch.tensor(encoding["token_type_ids"], dtype=torch.long)
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

#Evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    precision = precision_score(labels, preds, average='macro')
    recall = recall_score(labels, preds, average='macro')
    f1 = f1_score(labels, preds, average='macro')
    accuracy = accuracy_score(labels, preds)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }
#param
def print_model_summary(model):
    print("📄 Model Architecture:\n")
    print(model)

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"\n🔢 Total parameters: {total_params}")
    print(f"🛠️ Trainable parameters: {trainable_params}")

#Loop through models
for config in model_configs:
    print(f"\n Training & Evaluating: {config['name']}")
    
    tokenizer = AutoTokenizer.from_pretrained(config["model_name"])
    model = ImprovedPOSAwareBanglaBERT(model_name=config["model_name"])
    pos_encoder = POSTagEncoder()
    print_model_summary(model)
    #Build dataset
    train_dataset = POSAwareDataset(X_train.tolist(), y_train.tolist(), tokenizer, 128, pos_encoder)
    val_dataset = POSAwareDataset(X_val.tolist(), y_val.tolist(), tokenizer, 128, pos_encoder)
    test_dataset = POSAwareDataset(X_test.tolist(), y_test.tolist(), tokenizer, 128, pos_encoder)
    
    #Training arguments
    training_args = TrainingArguments(
        output_dir=f"./results/{config['name'].replace(' ', '_')}",
        num_train_epochs=15,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        logging_dir=f"./logs/{config['name'].replace(' ', '_')}",
        logging_steps=100,
        eval_strategy="steps",
        eval_steps=300,
        save_strategy="steps",
        save_steps=300,
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model='f1',
        greater_is_better=True,
        weight_decay=0.01,
        learning_rate=1e-5,
        lr_scheduler_type="linear",
        warmup_ratio=0.1,
        fp16=True,
        report_to=None
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    trainer.train()

    #Evaluate on validation
    print(f"\n📊 Validation Results for {config['name']}:")
    val_results = trainer.evaluate()
    print(val_results)

    #Evaluate on test
    print(f"\n📦 Evaluating on Test Set...")
    test_results = trainer.evaluate(test_dataset)
    print(f"📊 Test Results: {test_results}")

    #Classification report
    predictions = trainer.predict(test_dataset)
    y_pred = np.argmax(predictions.predictions, axis=1)
    y_true = predictions.label_ids
    print(f"\n📄 Classification Report for {config['name']}:\n")
    print(classification_report(y_true, y_pred))

    # free memory and delete checkpoints
    del model, tokenizer, trainer
    torch.cuda.empty_cache()
    gc.collect()
    shutil.rmtree(f"./results/{config['name'].replace(' ', '_')}", ignore_errors=True)



 Training & Evaluating: MuRIL


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/53.9M [00:00<?, ?B/s]

📄 Model Architecture:

ImprovedPOSAwareBanglaBERT(
  (bert): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(29898, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
           

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
300,0.775100,0.667081,0.580347,0.549042,0.507384,0.406662
600,0.648700,0.525940,0.731792,0.726926,0.715876,0.718936
900,0.518600,0.460013,0.784971,0.779394,0.781692,0.780385
1200,0.469600,0.444924,0.806936,0.803530,0.810420,0.804727
1500,0.426400,0.424424,0.816185,0.811454,0.816526,0.813164
1800,0.386300,0.412438,0.820809,0.816441,0.822396,0.818181
2100,0.412800,0.396939,0.825434,0.821097,0.827140,0.822873
2400,0.362200,0.416370,0.821965,0.817332,0.822643,0.819093
2700,0.323700,0.418138,0.827746,0.822989,0.827633,0.824701
3000,0.342000,0.408883,0.833526,0.828884,0.830745,0.829739



📊 Validation Results for Bangla Electra:


{'eval_loss': 0.4088830053806305, 'eval_accuracy': 0.8335260115606936, 'eval_precision': 0.828883945124212, 'eval_recall': 0.830745103200193, 'eval_f1': 0.829739305397603, 'eval_runtime': 1.8053, 'eval_samples_per_second': 479.15, 'eval_steps_per_second': 30.466, 'epoch': 8.314087759815243}

📦 Evaluating on Test Set...
📊 Test Results: {'eval_loss': 0.36189427971839905, 'eval_accuracy': 0.8464203233256351, 'eval_precision': 0.8420520106776888, 'eval_recall': 0.8475569409126952, 'eval_f1': 0.8439481082545812, 'eval_runtime': 1.8607, 'eval_samples_per_second': 465.42, 'eval_steps_per_second': 29.559, 'epoch': 8.314087759815243}

📄 Classification Report for Bangla Electra:

              precision    recall  f1-score   support

           0       0.89      0.84      0.86       501
           1       0.80      0.85      0.82       365

    accuracy                           0.85       866
   macro avg       0.84      0.85      0.84       866
weighted avg       0.85      0.85      0.85      